In [86]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy 
import pickle

from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

#Change this line based on your PC configurations.
path = "C:/Users/Arikhan Arik/Desktop/American Express - Default Prediction/Data/"

# Change this line based on your PC configurations.
model_path = "C:/Users/Arikhan Arik/Desktop/American Express - Default Prediction/Model/"


In [87]:
train_df = pd.read_parquet(path + "train_data.parquet")

# test_data1 = pd.read_parquet("C:/Users/Arikhan Arik/Desktop/American Express - Default Prediction/Data/test_data1.parquet")
# test_data2 = pd.read_parquet("C:/Users/Arikhan Arik/Desktop/American Express - Default Prediction/Data/test_data2.parquet")

In [88]:
train_df.head(10)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,...,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,...,NaN,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,...,NaN,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,...,NaN,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,...,NaN,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827
5,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-08-04,0.945801,0.001746,0.007866,1.004883,0.004219,0.110962,0.009857,0.009865,...,NaN,NaN,NaN,0.001925,0.008598,0.004528,NaN,0.000674,0.002222,0.002884
6,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-09-18,0.940918,0.002182,0.018860,1.007812,0.004509,0.103333,0.006603,0.000783,...,NaN,NaN,NaN,0.001336,0.004360,0.009384,NaN,0.007729,0.007660,0.002224
7,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-10-08,0.914551,0.003029,0.014328,1.000000,0.000263,0.108093,0.009529,0.007835,...,NaN,NaN,NaN,0.002398,0.008453,0.005554,NaN,0.001831,0.009613,0.007385
8,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-11-20,0.950684,0.009895,0.016891,1.003906,0.001789,0.102783,0.002520,0.009819,...,NaN,NaN,NaN,0.009743,0.003967,0.007942,NaN,0.008720,0.004368,0.000996
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-04,0.868652,0.001082,0.001930,1.007812,0.001772,0.100464,0.004627,0.006073,...,NaN,NaN,NaN,0.003611,0.009605,0.007267,NaN,0.008766,0.004753,0.009071


In [89]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(177), object(13)
memory usage: 4.2+ GB


In [90]:
# Apply memory reduction for df_test since the data is way too big for memory in default settings.
for column in train_df.select_dtypes(include=['int']).columns:
    if train_df[column].max() <= np.iinfo(np.int8).max and train_df[column].min() >= np.iinfo(np.int8).min:
        train_df[column] = train_df[column].astype('int8')
    elif train_df[column].max() <= np.iinfo(np.int16).max and train_df[column].min() >= np.iinfo(np.int16).min:
        train_df[column] = train_df[column].astype('int16')
    elif train_df[column].max() <= np.iinfo(np.int32).max and train_df[column].min() >= np.iinfo(np.int32).min:
        train_df[column] = train_df[column].astype('int32')

for column in train_df.select_dtypes(include=['float']).columns:
    if train_df[column].max() <= np.finfo(np.float16).max and train_df[column].min() >= np.finfo(np.float16).min:
        train_df[column] = train_df[column].astype('float16')
    elif train_df[column].max() <= np.finfo(np.float32).max and train_df[column].min() >= np.finfo(np.float32).min:
        train_df[column] = train_df[column].astype('float32')

In [91]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(177), object(13)
memory usage: 4.2+ GB


In [92]:
# test_df = pd.concat([test_data1, test_data2])

In [93]:
# test_df.info()

In [94]:
# del test_data1
# del test_data2 

In [95]:
train_df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,...,NaN,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,...,NaN,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,...,NaN,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,...,NaN,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,...,NaN,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827


In [96]:
train_df['S_2'] = train_df['S_2'].str.replace("-", "").str[0:6]

In [97]:
null_percentages = train_df.isnull().sum() * 100 / len(train_df)
null_percentages.sort_values(ascending = False).head(30)

D_87     99.930127
D_88     99.891457
D_108    99.476846
D_111    99.433530
D_110    99.433530
B_39     99.391986
D_73     98.990211
B_42     98.707789
D_134    96.480146
D_135    96.480146
D_136    96.480146
D_137    96.480146
D_138    96.480146
R_9      94.349891
B_29     93.104594
D_106    90.213255
D_132    90.191055
D_49     90.137597
R_26     88.984717
D_76     88.746226
D_66     88.730733
D_42     85.694278
D_142    82.926577
D_53     73.842921
D_82     73.373406
D_50     56.809723
B_17     56.722874
D_105    54.622756
D_56     54.071581
S_9      53.035686
dtype: float64

In [98]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 190 entries, customer_ID to D_145
dtypes: float32(177), object(13)
memory usage: 4.2+ GB


In [99]:
# Drop the columns that contains more than 50% of null values.
train_df.drop(list(null_percentages.sort_values(ascending = False).head(30).index), axis = 1, inplace = True)

In [100]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 160 entries, customer_ID to D_145
dtypes: float32(148), object(12)
memory usage: 3.5+ GB


# Categorical data null analysis and conversion

In [101]:
# Find the columns that contains the categorical values.
cat_columns = list(train_df.select_dtypes('object').columns)

In [102]:
# Calculate and sort the null percentages for each column.
null_percentages = train_df[cat_columns].isnull().sum() * 100 / len(train_df)
null_percentages.sort_values(ascending = False).head(30)

D_64           3.931012
D_68           3.914036
D_114          3.194749
D_116          3.194749
D_117          3.194749
D_120          3.194749
D_126          2.111851
B_30           0.036446
B_38           0.036446
customer_ID    0.000000
S_2            0.000000
D_63           0.000000
dtype: float64

In [103]:
# Check the value frequencies for D_64 column.
train_df.D_64.value_counts()

O     2913244
U     1523448
R      840112
-1      37205
Name: D_64, dtype: int64

In [104]:
# Fill the Null values as NA, since it is a deliquency (minor crime, especially that committed by young people.) field.
train_df.D_64.fillna("NA", inplace = True)
train_df.D_64 = train_df.D_64.str.replace("-1", "NA")

In [105]:
# Normalized by the meaning of value counts.
train_df.D_64.value_counts()

O     2913244
U     1523448
R      840112
NA     254647
Name: D_64, dtype: int64

In [106]:
# Check the value frequencies for D_68 column.
train_df.D_68.value_counts()

6.0    2782455
5.0    1201706
3.0     484442
4.0     477187
2.0     220111
1.0     133122
0.0      15925
Name: D_68, dtype: int64

In [107]:
# The field is categorical, however we observe some numeric values. It might be because of many things.
# We could put -1 as representative for Null values.
train_df.D_68.fillna("-1", inplace = True)

In [108]:
# Check the value frequencies for D_68 column.
train_df.D_68.value_counts()

6.0    2782455
5.0    1201706
3.0     484442
4.0     477187
2.0     220111
-1      216503
1.0     133122
0.0      15925
Name: D_68, dtype: int64

In [109]:
# For the columns D_114, D_116, D_117, D_120 the null percentage is the same, which is interesting.
# Let's see if the null values are actually on the same rows. That would mean that those feature could be conditionally dependent.

train_df.D_114.value_counts()

1.0    3316478
0.0    2038257
Name: D_114, dtype: int64

In [110]:
train_df.D_114.fillna("-1", inplace = True)

In [111]:
train_df.D_114.value_counts()

1.0    3316478
0.0    2038257
-1      176716
Name: D_114, dtype: int64

In [112]:
train_df.D_116.value_counts()

0.0    5348109
1.0       6626
Name: D_116, dtype: int64

In [113]:
train_df.D_116.fillna("-1", inplace = True)

In [114]:
train_df.D_116.value_counts()

0.0    5348109
-1      176716
1.0       6626
Name: D_116, dtype: int64

In [115]:
train_df.D_117.value_counts()

-1.0    1456084
3.0     1166400
4.0     1138666
2.0      666808
5.0      459290
6.0      344520
1.0      122967
Name: D_117, dtype: int64

In [116]:
# Filling with -2 because we already have -1. I reckon -1 also has a special meaning, however it is the most frequent value.
# Therefore for now we will keep it as -1 and nulls will be -2. This could be changed at other iterations.
train_df.D_117.fillna("-2", inplace = True)

In [117]:
train_df.D_117.value_counts()

-1.0    1456084
3.0     1166400
4.0     1138666
2.0      666808
5.0      459290
6.0      344520
-2       176716
1.0      122967
Name: D_117, dtype: int64

In [118]:
train_df.D_120.value_counts()

0.0    4729723
1.0     625012
Name: D_120, dtype: int64

In [119]:
train_df.D_120.fillna("-1", inplace = True)

In [120]:
train_df.D_120.value_counts()

0.0    4729723
1.0     625012
-1      176716
Name: D_120, dtype: int64

In [121]:
train_df.D_126.value_counts()

1.0     4262414
0.0      891323
-1.0     260898
Name: D_126, dtype: int64

In [122]:
train_df.D_126.fillna("-2", inplace = True)

In [123]:
train_df.D_126.value_counts()

1.0     4262414
0.0      891323
-1.0     260898
-2       116816
Name: D_126, dtype: int64

In [124]:
train_df.B_30.value_counts()

0.0    4710663
1.0     763955
2.0      54817
Name: B_30, dtype: int64

In [125]:
train_df.B_30.fillna("-1", inplace = True)

In [126]:
train_df.B_30.value_counts()

0.0    4710663
1.0     763955
2.0      54817
-1        2016
Name: B_30, dtype: int64

In [127]:
train_df.B_38.value_counts()

2.0    1953232
3.0    1255315
1.0    1160047
5.0     444856
4.0     294917
7.0     259028
6.0     162040
Name: B_38, dtype: int64

In [128]:
train_df.B_38.fillna("-1", inplace = True)

In [129]:
train_df.B_38.value_counts()

2.0    1953232
3.0    1255315
1.0    1160047
5.0     444856
4.0     294917
7.0     259028
6.0     162040
-1        2016
Name: B_38, dtype: int64

In [130]:
customer_ids = train_df['customer_ID']
train_df.drop("customer_ID", axis = 1,inplace = True)

In [131]:
train_df.D_63.value_counts()

CO    4119621
CR     930133
CL     438390
XZ      25786
XM      10556
XL       6965
Name: D_63, dtype: int64

In [132]:
cat_columns.remove("customer_ID")
cat_columns

['S_2',
 'D_63',
 'D_64',
 'D_68',
 'B_30',
 'B_38',
 'D_114',
 'D_116',
 'D_117',
 'D_120',
 'D_126']

In [133]:
# The imputation done in a very easy way. Therefore, for the next iterations we could change our approach based on more analysis.
# Now, it is time for the categorical --> numeric conversion.
train_df = pd.get_dummies(train_df)

In [134]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Columns: 212 entries, P_2 to D_126_1.0
dtypes: float32(148), uint8(64)
memory usage: 3.4 GB


# Numeric value imputation

In [135]:
# Find the columns that contains the categorical values.
numeric_columns = list(train_df.select_dtypes('float32').columns)

In [136]:
numeric_columns

['P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'B_6',
 'B_7',
 'B_8',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'B_13',
 'R_5',
 'D_58',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_65',
 'B_16',
 'B_18',
 'B_19',
 'B_20',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'P_4',
 'D_74',
 'D_75',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'S_18',
 'D_86',
 'R_17',
 'R_18',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 'S_20',
 'R_20',
 'R_21',
 'B_33',
 'D_89',
 'R_22',
 'R_23',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'R_24',
 'R_25',
 'D_96',
 'S_22',
 'S_23',
 'S_24',
 'S_25',
 'S_26',

In [137]:
# Let's find and impute the missing values for each numeric values now to make the data "ready".
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

train_df[numeric_columns] = pd.DataFrame(imp_mean.fit_transform(train_df[numeric_columns]), columns = numeric_columns)

In [138]:
# Test that we have no null values left, both for the numerical and categorical features. We expect max null values as "0".
train_df.isna().sum().max()

0

In [139]:
train_df.head(5)

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_117_4.0,D_117_5.0,D_117_6.0,D_120_-1,D_120_0.0,D_120_1.0,D_126_-1.0,D_126_-2,D_126_0.0,D_126_1.0
0,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,0.154684,0.000630,...,1,0,0,0,1,0,0,0,0,1
1,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,0.154684,0.002525,...,1,0,0,0,1,0,0,0,0,1
2,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,0.154684,0.007607,...,1,0,0,0,1,0,0,0,0,1
3,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,0.154684,0.006405,...,1,0,0,0,1,0,0,0,0,1
4,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,0.154684,0.007732,...,1,0,0,0,1,0,0,0,0,1


In [140]:
train_labels = pd.read_csv(path + "train_labels.csv")

In [141]:
train_labels

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [142]:
# Test that all of the customerIDs are actually unique. In that case, we need max number of "1".
train_labels.customer_ID.value_counts().max()

1

In [143]:
# Construct another dataframe for the labels with customer ID values. 
labels = pd.DataFrame()
labels['customer_ID'] = customer_ids

In [144]:
# Left join the 
labels = labels.merge(train_labels, left_on='customer_ID', right_on='customer_ID', how = 'left')

In [145]:
labels

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,0


In [146]:
# Assign the target values respective to the customerID's and to be continued from here with the XGBoost model and parameter tuning.
train_df['target'] = labels['target']
train_df['customer_ID'] = labels['customer_ID']

In [147]:
train_df

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_117_6.0,D_120_-1,D_120_0.0,D_120_1.0,D_126_-1.0,D_126_-2,D_126_0.0,D_126_1.0,target,customer_ID
0,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,0.154684,0.000630,...,0,0,1,0,0,0,0,1,0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
1,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,0.154684,0.002525,...,0,0,1,0,0,0,0,1,0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
2,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,0.154684,0.007607,...,0,0,1,0,0,0,0,1,0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
3,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,0.154684,0.006405,...,0,0,1,0,0,0,0,1,0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
4,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,0.154684,0.007732,...,0,0,1,0,0,0,0,1,0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,0.979492,0.416016,0.020813,0.828125,0.003487,0.090759,0.005341,0.025146,0.014229,0.002178,...,0,0,1,0,0,0,0,1,0,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...
5531447,0.984863,0.296631,0.007210,0.812500,0.005905,0.079895,0.002243,0.023697,0.009468,0.005219,...,0,0,1,0,0,0,0,1,0,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...
5531448,0.982910,0.444092,0.013153,0.815430,0.003456,0.100525,0.002111,0.012344,0.006592,0.002350,...,0,0,1,0,0,0,0,1,0,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...
5531449,0.969727,0.442627,0.009857,1.003906,0.005116,0.101807,0.009933,0.008575,0.014236,0.006508,...,0,0,1,0,0,0,0,1,0,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...


# Division of train data in to train/validation sets.

In [148]:
#lets see how many unique customer we have.
train_df.customer_ID.unique().shape[0]

458913

In [149]:
# We can divide the customers by 80/20 pareto, or we can do the same for the row numbers.
# I will divide based on customer IDs as starters. We can change it to row numbers afterwards.

In [150]:
# find the divident point.
print("Get the first %d as the train set" %(train_df.customer_ID.unique().shape[0] * 80 / 100))
print("Get the last %d as the validation set" %(train_df.customer_ID.unique().shape[0] * 20 / 100))

train_set_shape = train_df.customer_ID.unique().shape[0] * 80 / 100

Get the first 367130 as the train set
Get the last 91782 as the validation set


In [151]:
# Get your train and validation sets.
train_set = train_df[train_df.customer_ID.isin(list(train_df.customer_ID.unique())[0:int(train_set_shape)])]
val_set = train_df[~train_df.customer_ID.isin(list(train_df.customer_ID.unique())[0:int(train_set_shape)])]

In [152]:
# Double check the number of customers. Seems correct.
train_set.customer_ID.unique().shape[0]

367130

In [153]:
# There is one number of difference because when we divide we *floored* the double values in to integer.
val_set.customer_ID.unique().shape[0]
val_set_cust_ID = val_set.customer_ID

In [154]:
# Let's get our train_x, train_y, val_x and val_y and delete unnecessary data to relax the memory.

train_x = train_set.drop(["customer_ID", "target"], axis = 1)
train_y = train_set.target
val_x = val_set.drop(["customer_ID", "target"], axis = 1)
val_y = val_set.target

del train_df
del train_set
del val_set

In [155]:
# Set the indexes again for the val_x and val_y because we have divided the data and didn't set the indexes. 
val_x.reset_index(drop=True, inplace = True)
val_y.reset_index(drop=True, inplace = True)

# Insert your ML Model here.

In [ ]:
# Define your model and train with train_x, train_y
# I will go for GBM/XGBoost (https://xgboost.readthedocs.io/en/stable/index.html)
model = XGBClassifier(booster = 'dart', 
                      eta = 0.1, 
                      gamma = 0.3, 
                      max_depth = 5, 
                      sampling_method = 'gradient_based',
                      reg_lambda = 0.2,
                      alpha = 0.2)

model.fit(train_x, train_y)

# make predictions for validation data
val_hat = model.predict(val_x)
val_hat_proba = model.predict_proba(val_x)

[07:51:47] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [83]:
# Save the model since it takes long time to train one model...
pickle.dump(model, open(model_path + "xgb_initial.pkl", "wb"))

In [157]:
#Custom Metric from competition providers.

def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [193]:
val_set_cust_ID.reset_index(drop=True, inplace = True)

In [194]:
train_labels = pd.read_csv(path + 'train_labels.csv')

train_labels = train_labels[train_labels.customer_ID.isin(val_set_cust_ID.unique())]

train_labels.set_index("customer_ID", inplace = True)

train_labels

,target
customer_ID,
cc8b10931bb8a34fefa8ce9dd70b0ff91a525b839569faefd94d1f7c8d77e27c,0
cc8b27a50901366262e6684b8f98e0127b7f713640d7f076138b6ab260e919e7,0
cc8b3bdc5a2ccc4a97c8c7bf9bc568eeb74b541cde5428c4888059906e961ac4,1
cc8b7e67b9d04662e4bfb3e86778b1bc4b0bc8e4bec9555efb6e31f97749f7f9,1
cc8b820d99341b60f83dcf637865a3c1e613a92072c18897e034c567e11581e4,0
...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0


In [214]:
val_hat_proba = model.predict_proba(val_x)
val_hat_proba

array([[0.55453813, 0.44546187],
       [0.53337723, 0.46662277],
       [0.53239393, 0.46760607],
       ...,
       [0.9962114 , 0.00378861],
       [0.9969147 , 0.00308529],
       [0.9958587 , 0.00414127]], dtype=float32)

In [228]:
val_hat_df = pd.DataFrame()
val_hat_df['customer_ID'] = val_set_cust_ID
val_hat_df['prediction'] = val_hat_proba[:, 0]
val_hat_df.set_index('customer_ID', inplace = True)

In [229]:
val_hat_df = (val_hat_df
          .groupby('customer_ID')
          .mean())

# Scale the mean P_2 by the max value and take the compliment
val_hat_df['prediction'] = 1.0 - (val_hat_df['prediction'] / val_hat_df['prediction'].max())

In [230]:
val_y_df = (val_y_df
          .groupby('customer_ID')
          .mean())

In [231]:
val_y_df

,target
customer_ID,
cc8b10931bb8a34fefa8ce9dd70b0ff91a525b839569faefd94d1f7c8d77e27c,0
cc8b27a50901366262e6684b8f98e0127b7f713640d7f076138b6ab260e919e7,0
cc8b3bdc5a2ccc4a97c8c7bf9bc568eeb74b541cde5428c4888059906e961ac4,1
cc8b7e67b9d04662e4bfb3e86778b1bc4b0bc8e4bec9555efb6e31f97749f7f9,1
cc8b820d99341b60f83dcf637865a3c1e613a92072c18897e034c567e11581e4,0
...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0


In [232]:
amex_metric(val_y_df, val_hat_df)

0.7087708933071069